In [1]:
import pandas as pd
import numpy as np
import librosa
import os
import glob
from tqdm import tqdm
import IPython.display
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")

import cosine_similarity as cs
import freq_dataset as fd
cos = cs.CosineSimilarity()
freq = fd.MakeFreqDataset()

s_path = "../sounds/"

In [2]:
!tree -d "../sounds/"

../sounds/
├── animal
│   ├── 닭 소리
│   ├── 양 소리
│   ├── 말 소리
│   ├── 곰 소리
│   ├── 벌 소리
│   ├── 개 소리
│   ├── 새 소리
│   ├── 소 소리
│   ├── 돼지 소리
│   ├── 사자 소리
│   ├── 염소 소리
│   ├── 오리 소리
│   ├── 물개 소리
│   ├── 모기 소리
│   ├── 개구리 소리
│   ├── 까마귀 소리
│   ├── 풀벌레 소리
│   ├── 부엉이 소리
│   ├── 고양이 소리
│   ├── 코끼리 소리
│   ├── cats_dog_add_noise
│   └── cats_dogs
├── human
│   ├── 코 고는 소리
│   ├── 하품 소리
│   ├── 영어리딩
│   ├── baby_crying
│   ├── coughing
│   ├── crying
│   ├── laughing
│   ├── moaning
│   ├── nose-blowing
│   ├── original_clean
│   ├── original_noise
│   ├── panting
│   ├── read_men
│   ├── read_women
│   ├── screaming
│   ├── sighing
│   ├── teeth-chattering
│   ├── teeth-grinding
│   ├── throat-clearing
│   └── tongue-clicking
├── life
│   ├── 칼 소리
│   ├── 총 소리
│   ├── 물 끓는 소리
│   ├── 옷 관련 소리
│   ├── 물 마시는 소리
│   ├── 문 여닫는 소리
│   ├── 물 내리는 소리
│   ├── 물 따르는 소리
│   ├── 물 떨어지는 소리
│   ├── 종이 소리
│   ├── 쓰는 소리
│   ├── 걷는 소리
│   ├── 동전 소리
│   ├── 타는 소리
│   ├── 심장 소리
│   ├── 써는 소리
│   ├── 치는 소리
│  

In [2]:
o_path = glob.glob(f'{s_path}animal/cats_dogs/*') + glob.glob(f'{s_path}animal/cats_dog_add_noise/*')
path_ls = [file for file in o_path if file.endswith(".mp3") or file.endswith(".wav")]
name_ls = [name.split('/')[-1][:-4] for name in path_ls]
label_ls = [name.split('/')[-1][:3] for name in path_ls]
info_df1 = pd.DataFrame()
info_df1['path'] = path_ls
info_df1['label'] = label_ls
info_df1['name'] = name_ls
info_df1['noise'] = ['clean' if path.split("/")[-2]=='cats_dogs' else 'noise' for path in path_ls]

o_path = glob.glob(f'{s_path}human/original_clean/*')+glob.glob(f'{s_path}human/original_noise/*')+glob.glob(f'{s_path}human/영어리딩/*')
path_ls = [file for file in o_path if file.endswith(".mp3") or file.endswith(".wav")]
name_ls = [name.split('/')[-1][:-4] for name in path_ls]
label_ls = [name.split('/')[-1][:3] for name in path_ls]
info_df2 = pd.DataFrame()
info_df2['path'] = path_ls
info_df2['label'] = 'human'
info_df2['name'] = name_ls
info_df2['noise'] = [path.split("/")[-2].split('_')[-1] for path in path_ls]

o_path = glob.glob(f'{s_path}nature/*/*')
path_ls = [file for file in o_path if file.endswith(".wav")]
name_ls = [name.split('/')[-1][:-4] for name in path_ls]
info_df3 = pd.DataFrame()
info_df3['path'] = path_ls
info_df3['label'] = 'nature'
info_df3['name'] = name_ls
info_df3['noise'] = [path.split("/")[-2] for path in path_ls]

final_df = pd.concat([info_df1, info_df2, info_df3])
final_df.reset_index(drop=True, inplace=True)
final_df

,path,label,name,noise
0,../sounds/animal/cats_dogs/cat_119.wav,cat,cat_119,clean
1,../sounds/animal/cats_dogs/dog_barking_100.wav,dog,dog_barking_100,clean
2,../sounds/animal/cats_dogs/cat_54.wav,cat,cat_54,clean
3,../sounds/animal/cats_dogs/cat_15.wav,cat,cat_15,clean
4,../sounds/animal/cats_dogs/cat_30.wav,cat,cat_30,clean
...,...,...,...,...
1883,../sounds/nature/풍경 소리/1-38559-A-14.wav,nature,1-38559-A-14,풍경 소리
1884,../sounds/nature/풍경 소리/3-129678-A-13.wav,nature,3-129678-A-13,풍경 소리
1885,../sounds/nature/풍경 소리/4-181362-A-13.wav,nature,4-181362-A-13,풍경 소리
1886,../sounds/nature/풍경 소리/5-243459-A-14.wav,nature,5-243459-A-14,풍경 소리


In [3]:
o_path = list(set(glob.glob(f'{s_path}*/*/*')) - set(final_df['path']))
path_ls = [file for file in o_path if file.endswith(".mp3") or file.endswith(".wav")]
name_ls = [name.split('/')[-1][:-4] for name in path_ls]
label_ls = [name.split('/')[-3] for name in path_ls]
info_df = pd.DataFrame()
info_df['path'] = path_ls
info_df['label'] = label_ls
info_df['name'] = name_ls
info_df

,path,label,name
0,../sounds/human/read_women/1_0041.wav,human,1_0041
1,../sounds/human/read_women/sub100121a00147.wav,human,sub100121a00147
2,../sounds/human/laughing/ONWC_12_2_0_15_0_0_0.wav,human,ONWC_12_2_0_15_0_0_0
3,../sounds/human/read_men/jYYHDnEnnDY.0337.wav,human,jYYHDnEnnDY.0337
4,../sounds/human/read_men/3LR5IpiEvSc.0595.wav,human,3LR5IpiEvSc.0595
...,...,...,...
20611,../sounds/human/read_women/3_1857.wav,human,3_1857
20612,../sounds/human/read_men/3LR5IpiEvSc.0279.wav,human,3LR5IpiEvSc.0279
20613,../sounds/human/read_women/4_0071.wav,human,4_0071
20614,../sounds/life/타자치는 소리/1-94231-B-32.wav,life,1-94231-B-32


In [9]:
result_df = freq.make_frequency_df(tracks_df=info_df[:1000])
result_df.to_csv("../files/freq_dataset_1030_1.csv", encoding='utf-8-sig')
result_df

100%|██████████| 1000/1000 [15:31<00:00,  1.07it/s]


,name,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,sub100121a00011,0.336616,0.114689,0.010428,0.000048,1451.930283,6.484439e+05,1530.681115,141146.870036,2748.028232,...,-10.600484,40.416233,3.792123,47.504089,-2.852186,41.652531,-10.151857,49.800541,-4.155911,33.245262
1,4_1804,0.349974,0.115671,0.071556,0.005003,1886.968786,7.981058e+05,2198.066341,376942.061914,3874.170552,...,-2.772513,103.305740,0.752794,187.755066,6.228756,382.168365,2.582274,264.174591,8.121996,407.989471
2,2_0582,0.453871,0.125051,0.053803,0.003725,2579.420836,1.917635e+06,2449.378299,668818.714025,5065.271935,...,-3.558047,89.365005,1.331199,85.591690,-1.906000,182.656799,-1.155324,137.115402,3.753052,175.855789
3,nVToV-zn_C4.0167,0.386196,0.098672,0.079364,0.002621,1395.392919,8.128638e+05,1804.015140,250465.595729,2548.359590,...,-3.805040,44.126900,-1.421340,46.197392,-8.402034,85.609261,1.512769,39.209358,-4.529726,48.168972
4,3_0098,0.367353,0.120741,0.057042,0.002815,2222.788343,1.377473e+06,2187.032764,406679.891890,4309.059553,...,-3.401388,52.476299,-3.612014,65.812317,-3.405300,83.709198,-4.685134,124.898666,0.612162,183.855560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3LR5IpiEvSc.0051,0.433374,0.112533,0.095082,0.006839,1499.537955,8.495744e+05,1907.211231,576582.343082,3173.175684,...,0.509220,52.726891,-1.578923,46.290207,-0.637434,48.351837,-4.546787,76.598717,2.767131,44.132244
996,2_0300,0.406901,0.121576,0.070486,0.005871,2356.942845,1.728370e+06,2280.955551,649674.343343,4608.307343,...,-4.035268,254.251663,2.287196,207.022430,3.765143,247.752151,-3.465316,207.531647,1.093189,198.785934
997,3_4961,0.370844,0.117149,0.057016,0.003028,2064.218625,1.193002e+06,2067.288839,481359.771187,3976.660600,...,-7.874800,76.697395,-2.068999,73.170700,-4.279459,132.122681,-7.575491,129.428864,-0.007831,195.386124
998,1-31251-A-6,0.336741,0.096823,0.087378,0.001978,2053.880169,5.724047e+04,2016.902326,71672.272303,3785.357666,...,-13.704986,42.812336,-18.722424,101.847107,-10.141923,43.853607,-11.798749,43.702145,-6.341482,34.732174


In [10]:
result_df2 = freq.make_frequency_df(tracks_df=info_df[1000:2000])
result_df2.to_csv("../files/freq_dataset_1030_2.csv", encoding='utf-8-sig')
result_df2

100%|██████████| 1000/1000 [14:11<00:00,  1.17it/s]


,name,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,3_1620,0.341029,0.110828,0.066629,0.003163,2002.994288,1.025490e+06,2094.799217,638348.533487,4126.594755,...,-1.859747,66.327255,-4.512108,76.476692,-3.908977,115.974327,-2.810832,109.849228,-1.078182,191.142914
1,5-197913-A-18,0.504139,0.073042,0.084229,0.008524,3268.302880,6.067673e+05,2893.680754,70048.051334,6854.536947,...,4.417226,30.831367,2.949365,28.192635,6.450665,29.628138,2.424235,17.597157,8.209786,19.652462
2,4_5620,0.437639,0.119582,0.050146,0.002710,2485.371869,1.735153e+06,2384.827353,503915.486194,4854.915427,...,-4.163858,102.647949,2.683589,160.766296,0.052952,225.260620,1.980845,257.286987,4.302336,189.718124
3,2_0043,0.363968,0.118468,0.067698,0.003560,2207.232879,1.673612e+06,2246.888824,401608.184783,4201.045110,...,-3.138959,156.162964,-1.179121,153.223740,-2.027592,190.546448,-3.986025,178.829208,4.408886,265.658844
4,GDYN_Punching_Perc_RAW_SH - 5,0.609076,0.054526,0.028211,0.002629,3485.345385,3.880387e+05,3251.582482,153876.814968,7610.264648,...,0.582520,14.674466,-0.480970,19.714584,2.721480,17.026949,0.827474,10.838694,3.947678,27.774437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,4_3640,0.397912,0.117156,0.057778,0.003255,2134.891504,1.549800e+06,2217.770105,450637.768736,4284.793069,...,-4.907514,91.849716,-3.667121,141.995941,-7.545836,152.636627,-4.526711,90.011887,-3.067833,113.997177
996,4_3988,0.348740,0.115749,0.057317,0.002379,2039.407406,1.178266e+06,2116.283463,482672.728169,4030.423777,...,-5.138658,68.412254,-3.651733,128.886292,-4.103089,176.584229,-3.393305,184.598877,0.095063,185.178955
997,jYYHDnEnnDY.0374,0.386083,0.087799,0.037888,0.001472,1385.139816,6.729980e+05,1770.623294,148610.952673,2641.147260,...,-0.042593,50.959831,4.157257,35.684231,-3.216464,36.775780,-6.326979,42.414982,1.535098,49.108395
998,4_1330,0.366131,0.115638,0.061361,0.002998,2340.358704,1.335346e+06,2273.928906,353867.888322,4520.283778,...,-3.581886,101.693817,-3.416598,108.062523,0.398451,180.880386,1.169108,286.613983,4.486814,296.553192


In [11]:
result_df3 = freq.make_frequency_df(tracks_df=info_df[4000:7000])
result_df3.to_csv("../files/freq_dataset_1030_5.csv", encoding='utf-8-sig')
result_df3

100%|██████████| 3000/3000 [42:52<00:00,  1.17it/s]  


,name,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,3LR5IpiEvSc.0177,0.482273,0.097184,0.062990,0.003698,1501.220247,6.467478e+05,2019.596920,432011.806992,3350.904531,...,-3.182590,44.624985,-0.731451,47.249805,-3.906641,52.116692,0.517904,34.140415,1.166199,40.272800
1,bM32LJFpAus.0029,0.502053,0.104788,0.031970,0.001548,1549.497732,2.199749e+05,2188.794689,485118.517336,3929.176241,...,-5.141090,77.337204,4.334688,33.840183,-0.786906,53.255646,-0.841502,62.394466,5.055913,41.330494
2,3_0340,0.352158,0.118294,0.061866,0.002508,2095.237953,2.088902e+06,1974.745348,479353.898397,3797.798691,...,-6.072611,87.680122,-1.226990,165.432785,-1.589178,177.575424,-2.783567,230.967133,2.549736,255.401474
3,2_0937,0.444485,0.120425,0.058158,0.004061,2295.051979,1.714168e+06,2236.380281,558449.745480,4494.502315,...,-3.873602,65.058769,-1.352143,51.791103,-4.018868,59.029400,-5.232388,55.162960,-0.615538,94.878441
4,4-187769-A-14,0.630894,0.050308,0.018939,0.000006,1624.692464,3.284933e+05,1909.896534,51815.726650,3025.365194,...,7.448274,24.196312,-0.962357,21.540567,2.877285,30.524540,1.541334,25.225960,3.235263,17.525221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,4_0192,0.391943,0.122548,0.055427,0.002578,2259.129093,1.366778e+06,2298.135208,490067.948153,4526.897290,...,-5.652720,86.360847,0.194530,44.162083,-5.687586,89.222466,-3.616662,95.318314,6.152884,246.843353
2996,4_4092,0.316466,0.111346,0.070343,0.003808,2015.511481,1.261529e+06,2076.908042,322754.575130,3765.252577,...,-5.433063,74.510750,-1.437838,116.861221,-2.867368,223.951828,-1.687071,242.145416,6.134575,362.804474
2997,4_2279,0.374974,0.118695,0.059491,0.005036,2007.575890,7.276550e+05,2343.779720,549970.999285,4451.102389,...,4.559113,210.388351,1.832425,200.256348,2.956485,71.419403,-4.400940,70.502151,-1.337462,45.095436
2998,sub100121a00196,0.377782,0.115736,0.006702,0.000017,1422.045209,4.966660e+05,1552.750301,154667.658602,2862.954712,...,-8.243173,34.066540,-0.001023,31.825130,-5.167278,34.396233,-5.274334,36.646080,-3.862601,28.986240


In [5]:
result_df4 = freq.make_frequency_df(tracks_df=info_df[10000:15000])
result_df4.to_csv("../files/freq_dataset_1030_7.csv", encoding='utf-8-sig')
result_df4

100%|██████████| 5000/5000 [1:10:36<00:00,  1.18it/s]


,name,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,1_0582,0.340335,0.113292,0.074783,0.004285,2328.597524,1.783645e+06,2304.003247,280848.567907,4516.940440,...,-5.662121,58.988770,-2.652950,95.026970,-2.253654,140.177475,-4.409947,179.148483,7.136548,143.995712
1,3_2381,0.398356,0.111746,0.039902,0.002038,2386.304508,1.504276e+06,2277.990693,412030.258141,4565.526239,...,0.022282,97.254791,-0.515384,96.533310,0.765297,224.142563,3.110862,217.664185,8.264536,375.928314
2,4_1289,0.367370,0.107701,0.064155,0.003532,1998.264694,1.099370e+06,2100.548005,455038.380086,3879.373512,...,-3.878164,56.225147,-4.735794,50.251850,-4.678482,77.392471,-5.287274,59.880451,0.925621,156.431061
3,sub100120a00025,0.282276,0.106544,0.013329,0.000052,1690.723245,7.173455e+05,1640.204264,147018.190619,3240.202692,...,-12.903102,63.071243,6.813957,69.226013,-12.168688,87.747940,-9.231424,47.304798,-2.421633,75.615097
4,4_5018,0.340235,0.124601,0.057902,0.002479,1961.781601,9.450467e+05,2109.515210,488610.738575,3793.505859,...,-1.877638,324.455933,5.371761,346.004608,2.237386,342.692322,0.379577,326.173096,2.206429,345.647003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3_1458,0.388933,0.127210,0.061333,0.003051,2360.659249,1.701292e+06,2305.248837,537240.985475,4704.260648,...,-0.639324,176.942749,1.868214,191.910538,0.205772,135.825607,2.552959,207.198257,7.040284,252.376907
4996,3_2325,0.428313,0.118050,0.050771,0.003025,1923.382276,9.023747e+05,2150.022552,610669.193513,4140.689248,...,-0.974795,100.692566,2.833502,155.689438,-1.951519,215.419174,0.869867,139.937592,2.767365,158.260452
4997,3LR5IpiEvSc.0265,0.478037,0.103477,0.054443,0.003158,1691.401840,8.240372e+05,2106.196828,516736.701614,3808.879102,...,-1.222978,51.778267,-0.432940,50.543755,-5.354802,66.854622,-0.138665,38.784866,1.100998,41.639759
4998,4_3089,0.353852,0.119252,0.065665,0.003212,2079.115758,1.752566e+06,2058.645587,555034.527676,3960.989151,...,-7.309254,77.170410,-5.623875,84.043533,-5.501753,81.909431,-5.677797,156.227081,1.857784,335.906464


In [4]:
result_df5 = freq.make_frequency_df(tracks_df=info_df[17500:])
result_df5.to_csv("../files/freq_dataset_1030_9.csv", encoding='utf-8-sig')
result_df5

100%|██████████| 3116/3116 [42:41<00:00,  1.22it/s] 


,name,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,3_1478,0.388457,0.114971,0.065246,0.002970,1965.361041,1.012551e+06,2103.116194,629730.850065,4039.389648,...,-3.473721,59.022259,-2.233169,59.316025,-6.129309,59.817467,-5.653935,85.373184,2.878262,157.582672
1,nVToV-zn_C4.0452,0.493673,0.092270,0.044095,0.001568,1532.301706,1.015062e+06,2104.006635,319948.566241,3242.079021,...,-2.828835,58.033459,0.476432,54.734520,-1.369396,89.905968,3.035792,19.046307,0.965259,35.805214
2,3_3830,0.399523,0.108152,0.054875,0.003526,2239.194858,1.096921e+06,2329.295155,350897.820733,4446.235183,...,-0.446498,115.355156,2.428449,185.317993,4.125856,205.091507,7.295925,317.464966,7.250350,235.564499
3,bM32LJFpAus.0347,0.456743,0.103449,0.057802,0.003370,1429.537524,5.975702e+05,2010.103185,444634.484975,3164.375977,...,-1.627464,65.830246,0.413102,51.213310,-0.282894,51.420258,2.323312,41.051849,-0.383643,34.300476
4,jYYHDnEnnDY.0458,0.438276,0.104428,0.091078,0.007721,1596.610021,7.219398e+05,2085.946986,444991.483787,3521.499023,...,-3.513556,48.868687,-0.144490,35.870693,0.081486,51.061634,-1.342948,33.774136,-0.635863,31.693792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3111,3_1857,0.428166,0.124477,0.049003,0.002988,1903.216279,7.603458e+05,2103.297384,587146.357596,3854.154323,...,-2.429352,207.713593,3.067660,276.915497,-0.586698,275.021210,0.622931,185.145737,4.808550,278.950714
3112,3LR5IpiEvSc.0279,0.358710,0.115258,0.074597,0.005135,1446.905797,1.515618e+06,1667.238723,809170.240989,2862.570190,...,-3.061724,66.146324,-2.753470,41.580338,-1.902012,95.784996,-2.221099,47.314201,-1.535271,28.787563
3113,4_0071,0.357256,0.123874,0.068741,0.003765,2063.528164,8.262837e+05,2158.399375,429487.544746,4207.827148,...,-2.109149,128.838745,2.772648,317.867706,1.687012,268.067352,-1.242025,235.711304,-0.030317,155.156082
3114,1-94231-B-32,0.603441,0.057411,0.003489,0.000012,3482.488330,1.569393e+05,3018.181584,69322.023249,7387.035116,...,2.161985,23.358425,-1.187001,30.484703,2.914761,27.258562,-4.401916,24.144793,2.092800,19.923616
